# 3.1. 3 신경망에서의 단어 처리

In [ ]:
import numpy as np

C = np.array([[1, 0, 0, 0, 0, 0, 0]])
W = np.random.randn(7,3)
h = np.matmul(C,W)

print(h)

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks
from common.layers import MatMul

In [ ]:
import numpy as np
from common.layers import MatMul

c = np.array([[1, 0, 0, 0, 0, 0, 0]])
W = np.random.randn(7,3)
layer = MatMul(W)
h = layer.forward(c)
print(h)

# 3. 2. 1 CBOW 모델의 추론

In [ ]:
import numpy as np

from common.layers import MatMul

c0 = np.array([[1, 0, 0, 0, 0, 0, 0]])
c1 = np.array([[0, 0, 1, 0, 0, 0, 0]])

W_in = np.random.randn(7,3)
W_out = np.random.randn(3,7)

in_layer0 = MatMul(W_in)
in_layer1 = MatMul(W_in)
out_layer = MatMul(W_out)

h0 = in_layer0.forward(c0)
h1 = in_layer1.forward(c1)
h = 0.5 * (h0 + h1)
s = out_layer.forward(h)

print(s)

In [ ]:
from common.util import preprocess

text = 'You say goodbye and I say hello.'

corpus, word_to_id, id_to_word = preprocess(text)
print(corpus)

print(id_to_word)

In [ ]:
from common.util import *

text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)

contexts, target = create_contexts_target(corpus, window_size = 1)

vocab_size = len(word_to_id)
target = convert_one_hot(target, vocab_size)
contexts = convert_one_hot(contexts, vocab_size)

In [ ]:
import numpy as np

from common.layers import *

class SimpleCBOW:
    def __init__(self, vocab_size, hidden_size):
        V, H = vocab_size, hidden_size


        W_in = 0.01 * np.random.randn(V, H).astype('f')
        W_out = 0.01 * np.random.randn(H,V).astype('f')


        self.in_layer0 = MatMul(W_in)
        self.in_layer1 = MatMul(W_in)
        self.out_layer = MatMul(W_out)
        self.loss_layer = SoftmaxWithLoss()


        layers = [self.in_layer0, self.in_layer1, self.out_layer]
        self.params, self.grads = [], []

        for layer in layers:
            self.params += layer.params
            self.grads += layer.grads



        self.word_vecs_in = W_in
        self.word_vecs_out = W_out.T
        self.word_vecs_in_out = np.dot(W_in,W_out)
    def forward(self, contexts, target):
        h0 = self.in_layer0.forward(contexts[:,0])
        h1 = self.in_layer1.forward(contexts[:,1])
        h = (h0+h1)*0.5
        score = self.out_layer.forward(h)
        loss = self.loss_layer.forward(score, target)
        return loss


    def backward(self, dout=1):
        ds = self.loss_layer.backward(dout)
        da = self.out_layer.backward(ds)
        da *= 0.5
        self.in_layer1.backward(da)
        self.in_layer0.backward(da)
        return None





In [ ]:
from common.trainer import Trainer
from common.optimizer import Adam
from common.util import *


window_size = 1
hidden_size = 5
batch_size = 3

max_epoch = 1000

text = 'You say goodbay and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)


vocab_size = len(word_to_id)
contexts, target = create_contexts_target(corpus, window_size)
target = convert_one_hot(target, vocab_size)
contexts = convert_one_hot(contexts, vocab_size)

model = SimpleCBOW(vocab_size, hidden_size)

optimizer = Adam()

trainer = Trainer(model, optimizer)

trainer.fit(contexts, target, max_epoch, batch_size)
trainer.plot()

In [ ]:
word_vecs_in = np.round(model.word_vecs_in, 7)
word_vecs_out = np.round(model.word_vecs_out, 7)
word_vecs_in_out = np.round(model.word_vecs_in_out, 6)

print("입력층만 사용")
for word_id, word in id_to_word.items():
    print("%10s" %word, end='')
    print(word_vecs_in[word_id])
print("-"*100)

print("출력층만 사용")
for word_id, word in id_to_word.items():
    print("%10s" %word, end='')
    print(word_vecs_out[word_id])
print("-"*100)

print("입력, 출력층 사용")
for word_id, word in id_to_word.items():
    print("%10s" %word, end='')
    print(word_vecs_in_out[word_id])
print("-"*100)

# skip - Gram

In [ ]:
import numpy as np
from common.layers import MatMul, SoftmaxWithLoss


class SimpleSkipGram:
    def __init__(self, vocab_size, hidden_size):
        V, H = vocab_size, hidden_size

        
        W_in = 0.01 * np.random.randn(V, H).astype('f')
        W_out = 0.01 * np.random.randn(H, V).astype('f')

        
        self.in_layer = MatMul(W_in)
        self.out_layer = MatMul(W_out)
        self.loss_layer1 = SoftmaxWithLoss()
        self.loss_layer2 = SoftmaxWithLoss()

        
        layers = [self.in_layer, self.out_layer]
        self.params, self.grads = [], []
        for layer in layers:
            self.params += layer.params
            self.grads += layer.grads

        
        self.word_vecs_in = W_in
        self.word_vecs_out = W_out.T
        self.word_vecs_in_out = np.dot(W_in,W_out)

    def forward(self, contexts, target):
        h = self.in_layer.forward(target)
        s = self.out_layer.forward(h)
        l1 = self.loss_layer1.forward(s, contexts[:, 0])
        l2 = self.loss_layer2.forward(s, contexts[:, 1])
        loss = l1 + l2
        return loss

    def backward(self, dout=1):
        dl1 = self.loss_layer1.backward(dout)
        dl2 = self.loss_layer2.backward(dout)
        ds = dl1 + dl2
        dh = self.out_layer.backward(ds)
        self.in_layer.backward(dh)
        return None


In [ ]:
from common.trainer import Trainer
from common.optimizer import Adam
from common.util import *


window_size = 1
hidden_size = 5
batch_size = 3

max_epoch = 1000

text = 'You say goodbay and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)


vocab_size = len(word_to_id)
contexts, target = create_contexts_target(corpus, window_size)
target = convert_one_hot(target, vocab_size)
contexts = convert_one_hot(contexts, vocab_size)

model = SimpleSkipGram(vocab_size, hidden_size)

optimizer = Adam()

trainer = Trainer(model, optimizer)

trainer.fit(contexts, target, max_epoch, batch_size)
trainer.plot()

In [ ]:
word_vecs_in = np.round(model.word_vecs_in, 7)
word_vecs_out = np.round(model.word_vecs_out, 7)
word_vecs_in_out = np.round(model.word_vecs_in_out, 6)

print("입력층만 사용")
for word_id, word in id_to_word.items():
    print("%10s" %word, end='')
    print(word_vecs_in[word_id])
print("-"*100)

print("출력층만 사용")
for word_id, word in id_to_word.items():
    print("%10s" %word, end='')
    print(word_vecs_out[word_id])
print("-"*100)

print("입력, 출력층 사용")
for word_id, word in id_to_word.items():
    print("%10s" %word, end='')
    print(word_vecs_in_out[word_id])
print("-"*100)